# CLIP 을 활용해 책 표지 임베딩


In [10]:
!uv pip install git+https://github.com/openai/CLIP.git 

  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-4e_kwljs
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-req-build-4e_kwljs
  Resolved https://github.com/openai/CLIP.git to commit dcba3cb2e2827b402d2701e7e1c7d9fed8a20ef1
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached torchvision-0.24.1-cp310-cp310-manylinux_2_28_x86_64.whl.metadata (5.9 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.0/8.0 MB 8.5 MB/s  0:00:01 eta 0:00:01
  Created wheel for clip: filename=clip-1.0-py3-none-any.whl size=1369549 sha256=83add7418adf64a9b2e198b0783b14824ce35d040df2126effa0e8951af0b676
  Stored in directory: /tmp/pip-ephem-wheel-cache-jo4wqkih/wheels/da/2b/4c/d6691fa9597aac8bb85d2ac13b112deb897d5b50f5ad9a37e4
Successfully built clip
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3/3 [clip]1/3 [torchvision]


In [1]:
import clip
import torch
import pandas as pd
import numpy as np
from PIL import Image
from tqdm import tqdm
import os
import re
import warnings
from typing import Optional, List, Tuple

warnings.filterwarnings("ignore")

## CLIP 임베딩
책 표지 이미지를 CLIP으로 임베딩하여 평점 데이터에 추가

In [10]:
class CLIPImageEmbedder:

    def __init__(
        self,
        image_dir: str,
        data_path: str,
        model_name: str = "ViT-B/32",
        device: Optional[str] = None,
        batch_size: int = 32,
    ):
        self.image_dir = image_dir
        self.model_name = model_name
        self.batch_size = batch_size
        self.data_path = data_path

        if device is None:
            self.device = "cuda" if torch.cuda.is_available() else "cpu"
        else:
            self.device = device

        print(f"** device: {self.device}")

        # CLIP 모델 로드
        print(f"> CLIP 모델 ({model_name}) 로딩 시작")
        self.model, self.preprocess = clip.load(model_name, device=self.device)
        self.model.eval()

        print("> CLIP 모델 로딩 완료\n")

    # ISBN-이미지 매핑
    def create_image_mapping(self) -> pd.DataFrame:
        output_path = self.data_path + "book_image_map.csv"

        print("> ISBN-이미지 매핑 시작")

        if not os.path.exists(self.image_dir):
            raise FileNotFoundError(
                f"이미지 디렉토리를 찾을 수 없습니다: {self.image_dir}"
            )

        image_files = os.listdir(self.image_dir)
        rows = []
        failed_files = []

        for fname in tqdm(image_files, desc="  이미지 파일 스캔"):
            # mac 숨김파일 건너뛰기
            if fname.startswith("._") or fname.startswith("."):
                continue

            if fname.lower().endswith((".jpg")):
                match = re.match(r"(\d+)", fname)
                if match:
                    isbn = match.group(1)
                    isbn = isbn.zfill(10)
                    path = os.path.join(self.image_dir, fname)

                    if os.path.exists(path) and os.path.getsize(path) > 0:
                        rows.append({"isbn": isbn, "image_path": path})
                    else:
                        failed_files.append(fname)
                else:
                    failed_files.append(fname)

        df_img = pd.DataFrame(rows)
        df_img["isbn"] = df_img["isbn"].astype(str)

        original_len = len(df_img)
        df_img = df_img.drop_duplicates(subset=["isbn"], keep="first")

        print(f"\n> ISBN-이미지 매핑 결과:")
        print(f"- 총 이미지 개수: {len(image_files):,}개")
        print(f"- 매핑 성공: {len(df_img):,}개")
        print(f"- 중복 제거: {original_len - len(df_img):,}개")
        print(f"- 매핑 실패: {len(failed_files):,}개")

        if failed_files and len(failed_files) <= 10:
            print(f"\n...... 매핑 실패 파일 예시: {failed_files[:5]}")

        df_img.to_csv(output_path, index=False)

        print(f"> 매핑 파일 저장 완료 (경로: {output_path})\n")

        return df_img

    def _load_and_preprocess_batch(
        self, image_paths: List[str]
    ) -> Tuple[torch.Tensor, List[str]]:
        images = []
        valid_paths = []

        for path in image_paths:
            try:
                img = Image.open(path).convert("RGB")
                img_tensor = self.preprocess(img)
                images.append(img_tensor)
                valid_paths.append(path)

            except Exception as e:
                print(f"...... 이미지 로딩 실패: {path} - {str(e)}")
                continue

        if len(images) == 0:
            return None, []

        batch = torch.stack(images).to(self.device)
        return batch, valid_paths

    def generate_embeddings(
        self,
        resume: bool = True,
    ) -> pd.DataFrame:

        print("> CLIP 임베딩 생성 시작\n")

        mapping_csv = self.data_path + "book_image_map.csv"
        output_csv = self.data_path + "clip_image_embeddings.csv"

        if not os.path.exists(mapping_csv):
            raise FileNotFoundError(f"매핑 파일을 찾을 수 없습니다: {mapping_csv}")

        df = pd.read_csv(mapping_csv, dtype={"isbn": str})
        print(f"** 처리할 이미지: {len(df):,}개")

        # Resume 처리
        processed_isbns = set()
        if resume and os.path.exists(output_csv):
            existing_df = pd.read_csv(output_csv)
            processed_isbns = set(existing_df["isbn"].astype(str))

            df = df[~df["isbn"].astype(str).isin(processed_isbns)].reset_index(
                drop=True
            )

            print(
                f"** 기존 임베딩 발견: {len(processed_isbns):,}개 .... 남은 작업 {len(df):,}개 (이어서 처리 시작)"
            )

        if len(df) == 0:
            print("> 모든 이미지 처리 완료")
            return pd.read_csv(output_csv)

        all_embeddings = []
        all_isbns = []
        failed_count = 0

        num_batches = (len(df) + self.batch_size - 1) // self.batch_size

        with torch.no_grad():
            for i in tqdm(
                range(0, len(df), self.batch_size),
                total=num_batches,
                desc="  임베딩 생성",
            ):

                batch_df = df.iloc[i : i + self.batch_size]
                batch_paths = batch_df["image_path"].tolist()

                batch_images, valid_paths = self._load_and_preprocess_batch(batch_paths)

                if batch_images is None:
                    failed_count += len(batch_paths)
                    continue

                failed_in_batch = len(batch_paths) - len(valid_paths)
                failed_count += failed_in_batch

                embeddings = self.model.encode_image(batch_images)
                embeddings = embeddings / embeddings.norm(dim=-1, keepdim=True)
                embeddings_np = embeddings.cpu().numpy()

                for idx, path in enumerate(valid_paths):
                    original_idx = batch_df[batch_df["image_path"] == path].index[0]
                    isbn = batch_df.loc[original_idx, "isbn"]

                    all_embeddings.append(embeddings_np[idx])
                    all_isbns.append(isbn)

        emb_array = np.array(all_embeddings)
        emb_dim = emb_array.shape[1]

        # 결과 저장
        emb_cols = {f"clip_emb_{i}": emb_array[:, i] for i in range(emb_dim)}
        result_df = pd.DataFrame(emb_cols)
        result_df.insert(0, "isbn", all_isbns)
        result_df["isbn"] = result_df["isbn"].astype(str)

        print(f"\n>>> 임베딩 생성 결과:")
        print(f"- 성공: {len(result_df):,}개")
        print(f"- 실패: {failed_count:,}개")
        print(f"- dimension: {emb_dim}")

        if resume and os.path.exists(output_csv) and len(processed_isbns) > 0:
            existing_df = pd.read_csv(output_csv)
            result_df = pd.concat([existing_df, result_df], ignore_index=True)
            print(f"\n- 최종 총 개수: {len(result_df):,}개")

        result_df.to_csv(output_csv, index=False)
        print(f"> 임베딩 저장 완료 (경로: {output_csv})\n")

        return result_df

    def merge_with_ratings(
        self,
        ratings_csv: str,
        output_csv: str,
    ) -> pd.DataFrame:
        print("> 평점 데이터와 임베딩 병합 시작\n")

        embeddings_csv = self.data_path + "clip_image_embeddings.csv"

        ratings = pd.read_csv(ratings_csv)
        embeddings = pd.read_csv(embeddings_csv, dtype={"isbn": str})

        ratings["isbn"] = ratings["isbn"].astype(str).str.zfill(10)
        embeddings["isbn"] = embeddings["isbn"].astype(str).str.zfill(10)

        original_len = len(ratings)
        merged = ratings.merge(embeddings, on="isbn", how="left")

        emb_cols = [col for col in merged.columns if col.startswith("clip_emb_")]
        matched = merged[emb_cols[0]].notna().sum()
        missing = merged[emb_cols[0]].isna().sum()

        print(f">> 병합 결과")
        print(f"* 전체 평점 데이터: {original_len:,}건")
        print(f"* 이미지 매칭 성공: {matched:,}건 ({matched/original_len*100:.2f}%)")
        print(f"* 이미지 매칭 실패: {missing:,}건 ({missing/original_len*100:.2f}%)")

        if missing > 0:
            print(
                f"\n<결측치 처리> 매칭되지 않은 {missing:,}건은 임베딩을 0으로 채움......"
            )
            merged[emb_cols] = merged[emb_cols].fillna(0)

        merged.to_csv(output_csv, index=False)
        print(f"> 병합 결과 저장 (경로: {output_csv})\n")

        return merged

    def run_full_pipeline(
        self,
        mapping_csv: str,
        train_ratings_csv: str,
        test_ratings_csv: str,
        train_output_csv: str,
        test_output_csv: str,
        force_regenerate: bool = False,
    ) -> Tuple[pd.DataFrame, pd.DataFrame]:

        print("=" * 10)
        print("CLIP 이미지 임베딩 파이프라인 시작")
        print("=" * 10 + "\n")

        # Step 1: 이미지 매핑
        if force_regenerate or not os.path.exists(mapping_csv):
            self.create_image_mapping()
        else:
            print(f"> 기존 매핑 파일이 있음!!! ({mapping_csv})\n")

        # Step 2: 임베딩 생성
        embeddings_csv = self.data_path + "clip_image_embeddings.csv"
        if force_regenerate or not os.path.exists(embeddings_csv):
            self.generate_embeddings(resume=not force_regenerate)
        else:
            print(f"> 기존 임베딩 파일이 있음!!! ({embeddings_csv})\n")

        # Step 3: Train 병합
        print("[Train 데이터 병합]")
        train_merged = self.merge_with_ratings(train_ratings_csv, train_output_csv)

        # Step 4: Test 병합
        print("\n[Test 데이터 병합]")
        test_merged = self.merge_with_ratings(test_ratings_csv, test_output_csv)

        print("\n" + "=" * 10)
        print(">> 파이프라인 완료!")
        print("=" * 10 + "\n")

        return train_merged, test_merged


if __name__ == "__main__":

    IMAGE_DIR = "/data/ephemeral/home/sojin/data/images"  # TODO: 경로 확인
    DATA_PATH = "/data/ephemeral/home/sojin/data/v5/"  # TODO: 경로 확인

    embedder = CLIPImageEmbedder(
        image_dir=IMAGE_DIR,
        data_path=DATA_PATH,
        model_name="ViT-B/32",
        device="cuda",
        batch_size=32,
    )

    # 전체 파이프라인 한번에 실행 (train + test)
    train_merged, test_merged = embedder.run_full_pipeline(
        mapping_csv=DATA_PATH + "book_image_map.csv",
        train_ratings_csv=DATA_PATH + "train_ratings.csv",
        test_ratings_csv=DATA_PATH + "test_ratings.csv",
        train_output_csv=DATA_PATH + "train_ratings_with_clip.csv",
        test_output_csv=DATA_PATH + "test_ratings_with_clip.csv",
        force_regenerate=False,
    )

    print("\n>> 모든 작업 완료!")

** device: cuda
> CLIP 모델 (ViT-B/32) 로딩 시작
> CLIP 모델 로딩 완료

CLIP 이미지 임베딩 파이프라인 시작

> 기존 매핑 파일이 있음!!! (/data/ephemeral/home/sojin/data/v5/book_image_map.csv)

> CLIP 임베딩 생성 시작

** 처리할 이미지: 149,522개


  임베딩 생성: 100%|██████████| 4673/4673 [09:09<00:00,  8.50it/s]



>>> 임베딩 생성 결과:
- 성공: 149,522개
- 실패: 0개
- dimension: 512
> 임베딩 저장 완료 (경로: /data/ephemeral/home/sojin/data/v5/clip_image_embeddings.csv)

[Train 데이터 병합]
> 평점 데이터와 임베딩 병합 시작

>> 병합 결과
* 전체 평점 데이터: 306,795건
* 이미지 매칭 성공: 281,590건 (91.78%)
* 이미지 매칭 실패: 25,205건 (8.22%)

<결측치 처리> 매칭되지 않은 25,205건은 임베딩을 0으로 채움......
> 병합 결과 저장 (경로: /data/ephemeral/home/sojin/data/v5/train_ratings_with_clip.csv)


[Test 데이터 병합]
> 평점 데이터와 임베딩 병합 시작

>> 병합 결과
* 전체 평점 데이터: 76,699건
* 이미지 매칭 성공: 70,467건 (91.87%)
* 이미지 매칭 실패: 6,232건 (8.13%)

<결측치 처리> 매칭되지 않은 6,232건은 임베딩을 0으로 채움......
> 병합 결과 저장 (경로: /data/ephemeral/home/sojin/data/v5/test_ratings_with_clip.csv)


>> 파이프라인 완료!


>> 모든 작업 완료!
